In [1]:
from PIL import Image,ImageDraw
import json
import numpy as np
from matplotlib import pyplot as plt
from math import floor
Image.MAX_IMAGE_PIXELS = None
import urllib.request, json
import requests
from io import BytesIO
import os
from urllib.parse import urlparse

import logging
import boto3
from botocore.exceptions import ClientError

# check Pillow version number
import PIL
print('Pillow Version:', PIL.__version__)
import numpy as np
import os
from pathlib import Path
import re
import json
# load and show an image with Pillow
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import matplotlib.pyplot as plt

Pillow Version: 8.1.0


## Guidelines to run this notebook
***
1. This file assumes you have downloaded the entire code folder from <a href="https://drive.google.com/drive/folders/1O8HrV5hgWwTM5dNJzhEzOaQddZSZThp5?usp=sharing" target="_blank"> this google drive folder</a>. 

2. This files assumes Selwyn dataset in folder above 'code' folder. So please make sure to download the LINZ datatset before proceeding to run this notebook.<a href="https://drive.google.com/drive/u/0/folders/1SetuO2jiS2sEx105AjiRFYyJsKk9IpGV" target="_blank"> this google drive folder.</a>.

3. Please make sure to download config file in case you did not follow any of above steps <a href="https://drive.google.com/drive/folders/1p5eVDxIziHHWbvCaCwj-lBcrvggSsLyU?usp=sharing" target="_blank"> download config file from here.</a>.  

4. And please do modify config file to add your AWS S3 folders or comment `upload_file(...)` method in `mark_image` method in below cells.

In [2]:
print("This file assumes you have downloaded the entire code folder from https://drive.google.com/drive/folders/1O8HrV5hgWwTM5dNJzhEzOaQddZSZThp5?usp=sharing")

This file assumes you have downloaded the entire code folder from https://drive.google.com/drive/folders/1O8HrV5hgWwTM5dNJzhEzOaQddZSZThp5?usp=sharing


In [3]:
print("This files assumes LINZ dataset in folder above 'code' folder. So please make sure to download the LINZ datatset before proceeding to run this notebook.")

This files assumes LINZ dataset in folder above 'code' folder. So please make sure to download the LINZ datatset before proceeding to run this notebook.


In [4]:
print("Please make sure to download config file in case you did not follow any of above steps. https://drive.google.com/drive/folders/1p5eVDxIziHHWbvCaCwj-lBcrvggSsLyU?usp=sharing \n")
print("And please do modify config file to add your AWS S3 folders or comment upload_file(...) method in 'mark_image' method in below cells.")

Please make sure to download config file in case you did not follow any of above steps. https://drive.google.com/drive/folders/1p5eVDxIziHHWbvCaCwj-lBcrvggSsLyU?usp=sharing 

And please do modify config file to add your AWS S3 folders or comment upload_file(...) method in 'mark_image' method in below cells.


In [5]:
config_folder = os.path.join(r'give_your_repository_path\mturk-task-helper\config')
config_file_path = os.path.join(path, "config.json")
if not os.path.exists(config_file_path) or not os.path.exists(os.path.join(path, "config.json")) or not os.path.exists(os.path.join(path, "config", "config.json")):
    print("Please download config file. https://drive.google.com/drive/folders/1p5eVDxIziHHWbvCaCwj-lBcrvggSsLyU?usp=sharing \n")

Please download config file. https://drive.google.com/drive/folders/1p5eVDxIziHHWbvCaCwj-lBcrvggSsLyU?usp=sharing 



In [6]:
#defaults values
class_thumbnail_sizes = {'bus':           171,
                        'van_rv':        127,
                        'small':         101,
                        'specialized':   111,
                        'truck':         223,
                        'trailer_large': 219,
                        'trailer_small': 101,
                        'unknown':       127}
#marker_color = (200,200,200,255)
point_color =(0,255,0)
bb_color =(255,0,0)
tile_size = 400

classify_images_folder = "task1/"
bucketname=""

In [7]:
overlap = 0.2
stride = int(tile_size * (1-overlap))

## Upload file to AWS S3 bucket
### Upload one image to a given folder in the bucket

In [8]:
def upload_file(file_path:os.path, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """
    print("S3 parameters validation")
    assert file_path or bucket or object_name, "File name is empty"
    print(file_path, bucket, object_name)


    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_path

    # Upload the file
    this_response = None
    s3 = boto3.resource('s3')


    print("Connect to S3 API")
    try:
        with open(file_path, 'rb') as f:
            this_response = s3.Bucket('mturk-s3-cg').put_object(Key=object_name, Body=f,ContentType='image/png',ACL='public-read')
        print("Uploading to S3 is complete!", this_response)
    except ClientError as e:
        logging.error(e)
        return False
    return True



## Validate main image for alpha channel

In [9]:
def validate_imagefile(im):
    #check if alpha channel exists
    r,g,b,a = im.split()
    assert a,"Error: No Alpha channel for " +im
    image = np.asarray(im)
    rows, columns, channels = image.shape
    a = image[:, :, 3]
    count = np.count_nonzero(a==0) + np.count_nonzero(a==255)
    #print(count, (rows * columns))
    assert (count == (rows * columns)),"Error: Invalid values in Alpha channel for " +image_files[0]
    return True

In [10]:
# import math
# import colorsys
# def get_contrast_marker_color(r,g,b):
#     """ Polar Chromacity coordinates"""
#     red, green, blue = [(255,0,0), (0,255,0), (0,0,255)]
#     h,s,l = colorsys.rgb_to_hls(r/255.,g/255.,b/255.)
#     #r2,g2,b2 = colorsys.hls_to_rgb(h, 1, 0.5)
#     #h1,l1,s1 = colorsys.hls_to_rgb(r2,g2,b2)
#     hue2 = h * 360
#     #print(hue2)
#     marker_color = (0,0,0)
#     #print(hue2)
#     if hue2 > 60 and hue2 < 180:
#         marker_color = green
#     elif hue2 > 180 and hue2 < 300:
#         marker_color = blue
#     else:
#         marker_color = red
#     #print(marker_color, hue2)
    
#     return marker_color

In [11]:
import math
import colorsys
def get_marker_color(r,g,b):
    """ HLS and Polar Chromacity coordinates"""
    #define color ranges
    red, green, blue, cyan, yellow, fuschia = [(255,0,0), (0,255,0), (0,0,255), (0,255,255), (255,255,0), (255,0,255)]
    #convert to HLS
    h,s,l = colorsys.rgb_to_hls(r/255.,g/255.,b/255.)
    
    #find the angle (hue2 or h2) since h from hsl is in radians
    hue2 = (180+(h * 360))%360
    marker_color = (0,0,0)
    if hue2 > 0 and hue2 < 30 or hue2 > 330 and hue2 < 360:
        marker_color = red
    elif hue2 > 30 and hue2 <= 90:
        marker_color = yellow
    elif hue2 > 90 and hue2 <= 150:
        marker_color = green
    elif hue2 > 150 and hue2 < 210:
        marker_color = cyan
    elif hue2 > 210 and hue2 <=270:
        marker_color = blue
    else:
        marker_color = fuschia
    h1 = hue2/360
    R,G,B = tuple(round(val*255) for val in colorsys.hls_to_rgb(h1, 1-l,1))
    #print((R,G,B),marker_color)
    return fuschia

#### Crop corner images to remove negative numpy crop slices
Uncmment "#upload_file(image_file_path, bucketname, object_name)" line in following code  
if you want to upload to AWS S3 bucket name and AWS config keys are configured.

In [12]:
def crop_corner_images(im:Image, xy:tuple, size:int, marker_color:tuple, image_file_path:os.path,bucketname:str, folder_name:str, upload:bool):
    x,y = xy
    crop = im.crop((x-size, y-size, x+size, y+size))
    assert crop.size, "Invalid crop size."
    #plt.figure(figsize=(5, 5))
    #plt.imshow(np.asarray(crop))
    #print(crop.size)
    wc,hc = crop.size
    draw = ImageDraw.Draw(crop)
    w1,h1 = wc//2,hc//2
    #draw.line((w1, h1) + (w1+6,w1+6), fill=marker_color, width=1)
    #draw.line((w1, h1+6, w1+6, h1), fill=marker_color, width=1)
    draw.line((w1, 0)+ ( w1,hc), fill=marker_color,width=1)
    draw.line((0, h1)+ ( wc,h1), fill=marker_color, width=1)
    #draw.ellipse((w1+9,h1+9,w1+11, h1+11), fill=marker_color, width=3)
    crop.convert('RGB').save(image_file_path)
    w,h = crop.size
    
    #plt.figure(figsize=(5, 5))
    #plt.imshow(np.asarray(img))

    #get bucket folder and image name to upload to S3
    object_name = folder_name+os.path.basename(image_file_path)
    
    #print(object_name)
    if upload:
        upload_file(image_file_path, bucketname, object_name)

#### Crop and mark image slice
Uncomment "#upload_file(image_file_path, bucketname, object_name)" line in following code  
if you want to upload to AWS S3 bucket name and AWS config keys are configured.

In [13]:
def crop_mark_slice(imarr:np.array, marker_color:tuple, image_file_path:os.path, bucketname:str, folder_name:str, upload:bool):
    img = Image.fromarray(imarr)#.resize((400,400), Image.LANCZOS)
    #img = zoom_to(img, p[0], p[1], 3)
    draw = ImageDraw.Draw(img)
    #draw.line((0, 0) + img.size, fill=marker_color)
    #draw.line((0, img.size[1], img.size[0], 0), fill=marker_color)
    
    w,h = img.size

    #print(w,h)
    p = ( int(floor(w/2))-50, int(floor(h/2))-50, int(floor(w/2))+50, int(floor(h/2))+50 )
    #print(p)

    #draw boundingbox and point and line markers
    #draw.rectangle(p, fill=None, outline=bb_color,width=3)
    #draw.ellipse(( int(floor(w/2))-3, int(floor(w/2))-3, int(floor(h/2))+3, int(floor(h/2))+3 ), fill=point_color)
    
    w1,h1 = (w//2),(h//2)
    draw.line((w1, 0)+ ( w1,h), fill=marker_color, width=1)
    draw.line((0, h1)+ ( w,h1), fill=marker_color,width=1)
    
    #small plus sign marker
    #draw.line((w1, h1) + (w1+6,h1+6), fill=marker_color)
    #draw.line((w1, h1+6, w1+6, h1), fill=marker_color)
    
    img.convert('RGB').save(image_file_path)
    w,h = img.size
    
    #plt.figure(figsize=(5, 5))
    #plt.imshow(np.asarray(img))

    #get bucket folder and image name to upload to S3
    object_name = folder_name+os.path.basename(image_file_path)
    
    #print(object_name)
    if upload:
        upload_file(image_file_path, bucketname, object_name)

In [14]:
def extract_images_from_annotations(im, annotationsfile, outputdirname:str, cropsize:int, classify_images_folder:str, upload:bool, s3_url:str):
    imarray = np.asarray(im)
    assert imarray.shape, "Image from S3 bucket url could not be opened."
    #print(imarray.shape)
    size = int(cropsize/2)
    classes = {'small':'Small vehicles', 'bus':'Buses', 'specialized':'Specialized vehicles', 'trailer_large':'Large trailers', 'trailer_small':'Small trailers', 'truck':'Trucks', 'van_rv':'Vans and RVs', 'unknown':'Unknown vehicles', 'Not relevant vehicles':'Not relevant vehicles'}

    foldername = str(os.path.splitext(os.path.basename(annotationsfile))[0].replace("_annotations",""))
    #print(foldername)
    
    this_folder_filenames = []
    
    path =  os.getcwd()
    outputdirpath = os.path.join(path, outputdirname)
    if not os.path.exists(os.path.join(path, outputdirname)):
        os.mkdir(outputdirpath)
    p = os.path.join(outputdirpath, foldername)
    try:
        # Create target Directory
        os.mkdir(p)
        print("Directory " , foldername ,  " Created ") 
    except FileExistsError:
        print("Directory " , foldername ,  " already exists")
        
    # *******************Uncomment following lines to see main image preview ***********************    
    #plot Image
    #plt.figure(figsize=(5, 5))
    #plt.imshow(imarray)
    with open(annotationsfile) as json_file:
        data = json.load(json_file)
    assert data, "Invalid annotationsFile "+ str(annotationsfile)
    
    #print(data['locations'])
    for key, values in data['locations'].items():        
        #print(key, size)
        if key!="unknown":
            if len(values) != 0:
                for value in values:
                    x = int(value['x'])
                    y = int(value['y'])
                    
                    #validate crop 
                    crop = im.crop((x-size, y-size, x+size, y+size))
                    assert crop.size, "Invalid crop "+str(crop.size)
                    
                    W,H = im.size
                    
                    filename = foldername+"_"+str(x)+"-"+str(y)+".png"
                    #print(filename, W,H,classes[key], int(value['x']),int(value['y']))
                    img_url = ""
                    if upload:
                        img_url = s3_url+"task1/"+foldername+"/"+filename
                    this_folder_filenames.append({"foldername":foldername, "W" : str(W),"H": str(H), "x": int(value['x']), "y":int(value['y']),"class": classes[key], "image_url":img_url})
                    path = os.path.join(p, filename)
                    #get numpy array slice
                    crop_slice = np.s_[x-size:x+size, y-size:y+size]

                    #print(crop_slice, imarray.shape)

                    # get pixel color if you prefer contrast color
                    #r,g,b,a = im.getpixel((y,x))
                    #validate slice
                    marker_color = (200,200,200,255)
                    needs_padding = False
                    
                    # slice maybe for x,y locations around corners 
                    for slic in crop_slice:
                        if slic.start < 0 or (slic.stop-slic.start) < (2*size):
                            needs_padding = True
                            break

                    # either of slice locations are negative
                    if needs_padding:
                        crop_corner_images(im, (y,x), size, marker_color,path, bucketname, classify_images_folder+foldername+"/", upload)
                    # also slice may still be valid but less than crop size
                    else:
                        imarr = imarray[crop_slice]
                        h,w,_ = imarr.shape
                        
                        # less than crop size - use pillow crop
                        if h < cropsize or w < cropsize:
                            crop_corner_images(im, (y,x), size, marker_color,path, bucketname, classify_images_folder+foldername+"/", upload)
                        
                        #use numpy slice asis
                        else:
                            crop_mark_slice(imarray[crop_slice],marker_color, path, bucketname, classify_images_folder+foldername+"/", upload)
    return this_folder_filenames

In case if you are running this notebook, and if you do not have S3 configuration setup locally on your desktop/laptop, please comment upload_file method in mark_image method in below cell. Otherwise, do change bucketname, target folder name for classify task in config file i.e. https://drive.google.com/file/d/1LeAwCJ8JEtePZiSqvF1_FmmBefbdp3pR/view?usp=sharing 

In [15]:
path = Path(os.getcwd())
#get new folder's storage entry folder, there on, folders likely similar ?
filedir = os.path.join(r'give_your_repository_path\linz\LINZ\Final\001_selwyn-0125m-urban-aerial-photos-2012-2013\0001') # example
                       #"0001","0001","01 Final Annotation Galleries"
    
main_folders = [os.path.join(filedir,name) for name in os.listdir(filedir)]
results = dict()
this_directory = None
annotations_count = 0
file_class_count = dict()

#load json config
config_file_path = os.path.join(r'give_your_repository_path\mturk-task-helper\config\config_example.json')
if os.path.exists(config_file_path) :
    with open(config_file_path, "r") as jsonfile:
        config = json.load(jsonfile)
        print(config)
        assert config, "Config JSON has errors or is empty."

class_thumbnail_sizes = config["class_thumbnail_sizes"]

#marker_color = tuple(config["marker_color"])
point_color = tuple(config["point_marker_color"])
bb_color = config["bb_color"]
tile_size = config["tile_size"]
overlap = config["overlap"]
stride = int(tile_size * (1-overlap))
bucketname = config["bucketname"]
classify_images_folder = config["classify_images_folder"]
s3_url = config['s3_url']

{'overlap': 0.2, 'tile_size': 400, 'class_thumbnail_sizes': {'bus': 171, 'van_rv': 127, 'small': 101, 'specialized': 111, 'truck': 223, 'trailer_large': 219, 'trailer_small': 101, 'unknown': 127}, 'outline_marker_color': [200, 200, 200, 255], 'point_marker_color': [255, 0, 0], 'bb_color': [255, 0, 0], 'bucketname': 'mturk-s3-cg', 'classify_images_folder': 'test_folder/task1/', 'bb_images_folder': 'test_folder/task2/', 's3_url': 'https://mturk-s3-cg.s3.amazonaws.com/'}


In [16]:
#this is an example for generating slices with 300, 400 and 500 
len(main_folders)

6

Following cell walks through given folder in this I am only using single folder for one single main Image folder for generating images for 300, 400 and 500 respectively:
The example block of code -> extract_images_from_annotations(im,annotationsfile, "temp2_300", 300, ,classify_images_folder+"examples"+"/"+"300"+"/") slices subimages to 300*300 and generates cropped images and uploads to respective classifyImages folder 
Note that on AWS S3 storage I already created 3 folders "300", "400" and "500" respectively. In any case if you are running this notebook, and if you do not have S3 configuration setup locally on your desktop/laptop, please comment upload_file method in mark_image method. above.

In [17]:
classify_task_answers = []

for f in main_folders[:1]:
    this_key = None
    for root, dir, files in os.walk(f):
        #ignore ipynb_checkpoints folders
        if re.search(".ipynb_checkpoints",os.path.basename(Path(root))):
            continue
        if len(dir) != 0:
            #check if Annotation galleries exist
            galleries = [ d for d in dir if re.search("\d+\sFinal Annotation Galleries",str(d))]
            #check if main image file exists 000X_000X_image.png exists ?
            image_files = [ file for file in files if re.search("\d+(_)\d+(_)image.png",file)]
            annotation_files = [ file for file in files if re.search("\d+(_)\d+(_)annotations.json",file)]
        #if dir is empty, we continue
        #valid folders and files exists ?
        if len(galleries) > 0 and len(image_files) > 0 and image_files[0] and "0001_0001" in image_files[0] :
            #print(root)
            imagefilepath = Path(root).joinpath( image_files[0])
            #print(imagefilepath)
            annotationsfile = Path(root, annotation_files[0])
            im = Image.open(imagefilepath)
            #valid_image = validate_imagefile(im)
            
            #if invalid image, Assertion Errors are raised so this never moves to next statement
            this_image_classify_task_answers = extract_images_from_annotations(im,annotationsfile, "task1", 306,classify_images_folder+"examples/new"+"/"+"300"+"/", False, s3_url)
            print(this_image_classify_task_answers[0]['image_url'])
            classify_task_answers.extend(this_image_classify_task_answers)
            
            #if you want multiple sub-image sizes to be generated at a time for each main image
            #extract_images_from_annotations(im,annotationsfile, "temp3_400", 400,classify_images_folder+"examples/new"+"/"+"400"+"/", False, s3_url)


Directory  0001_0001  Created 



In [18]:
len(classify_task_answers)

11

### Generate cv file with answers

In [19]:
import pandas as pd
result = pd.DataFrame.from_dict(classify_task_answers)
result.to_csv(os.path.join(os.getcwd(),"selwyn_answers_classification_task1_test.csv"), index=False)

### Generate a different format of same answers

In [22]:
classes = {'small':'Small vehicles', 'bus':'Buses', 'specialized':'Specialized vehicles', 'trailer_large':'Large trailers', 'trailer_small':'Small trailers', 'truck':'Trucks', 'van_rv':'Vans and RVs', 'unknown':'Unknown vehicles', 'Not relevant vehicles':'Not relevant vehicles'}
all_answers = []
for f in main_folders[0:1]:
    this_key = None
    for root, dir, files in os.walk(f):
        #ignore ipynb_checkpoints folders
        if re.search(".ipynb_checkpoints",os.path.basename(Path(root))):
            continue
        if len(dir) != 0:
            #check if Annotation galleries exist
            galleries = [ d for d in dir if re.search("\d+\sFinal Annotation Galleries",str(d))]
            #check if main image file exists 000X_000X_image.png exists ?
            image_files = [ file for file in files if re.search("\d+(_)\d+(_)image.png",file)]
            annotation_files = [ file for file in files if re.search("\d+(_)\d+(_)annotations.json",file)]
        #if dir is empty, we continue
        #valid folders and files exists ?
        if len(galleries) > 0 and len(image_files) > 0 and image_files[0] :
            #print(root)
            imagefilepath = Path(root).joinpath( image_files[0])
            #print(imagefilepath)
            annotationsfile = Path(root, annotation_files[0])
            im = Image.open(imagefilepath)
            w,h = im.size
            #print((w,h),image_files[0], foldername )
            with open(annotationsfile) as json_file:
                data = json.load(json_file)
            for key, values in data['locations'].items():
                if len(values) != 0:
                    for value in values:
                        x = int(value['x'])
                        y = int(value['y'])
                        foldername = str(os.path.splitext(os.path.basename(annotationsfile))[0].replace("_annotations",""))
                        filename = foldername+"_"+str(x)+"-"+str(y)+".png"
                        k = classes[key]
                        #print(k)
                        all_answers.append({"image_name":image_files[0], "W" : str(w),"H": str(h), "x": int(value['x']), "y":int(value['y']),"class": classes[key], "image_url":"https://mturk-s3-cg.s3.amazonaws.com/"+"task1/batch100/"+foldername+"/"+filename})
            print("wh" , (str(w), str(h)),len(all_answers))

wh ('24174', '11141') 11
